In [0]:
!ls

sample_data


## Retriving Dataset directly from kaggle in Google-Colab

In [0]:
!pip install kaggle 

In [0]:
from google.colab import files
files.upload() #upload json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shujatali123","key":"76be1feee77195c0715b2ccb5ed0d093"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json 

In [0]:
!kaggle datasets download -d salader/dogs-vs-cats --force # directly downloading dataset

 97% 529M/545M [00:08<00:00, 58.7MB/s]
100% 545M/545M [00:08<00:00, 69.7MB/s]


In [0]:
!unzip dogs-vs-cats.zip #unzip files

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
from pathlib import Path
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout
# from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


# Data pre-processing and data augmentation

In [0]:
IMG_WIDTH, IMG_HEIGHT = 150, 150
TRAIN_DATA_DIR = 'train'
VALIDATION_DATA_DIR = 'test'
# MODEL_WEIGHTS_FILE = 'vgg16-xfer-tuned-weights.h5'
NB_TRAIN_SAMPLES = 20000
NB_VALIDATION_SAMPLES = 5000
NB_EPOCH = 20

In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = train_datagen.flow_from_directory(
        TRAIN_DATA_DIR,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=50,
        class_mode='categorical')


Found 20000 images belonging to 2 classes.


In [0]:
validation_generator = test_datagen.flow_from_directory(
        VALIDATION_DATA_DIR,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=50,
        class_mode='categorical')

Found 5000 images belonging to 2 classes.


# Using the pre-trained model VGG-16

In [0]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 1s 0us/step


In [0]:
model_vgg16_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

## Fine tunning the model with layer making Trainable parameters half

In [0]:
for layer in model_vgg16_conv.layers[:-4]:
    layer.trainable = False

In [0]:
model_vgg16_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

## Adding  of layers

In [0]:
input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
x = model_vgg16_conv(input)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax')(x)
model = Model(input=input, output=x)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  import sys


 ## Compile model and fit model

In [0]:
model.compile(loss='binary_crossentropy',
        optimizer=SGD(lr=1e-4, momentum=0.9),
        metrics=['accuracy'])

In [0]:
model.fit_generator(
        train_generator,
        callbacks = callbacks,
    
    
        steps_per_epoch=100,
        
        nb_epoch=NB_EPOCH,
        validation_data=validation_generator,
        validation_steps=100,
        
        verbose=2)

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., callbacks=[<keras.ca..., steps_per_epoch=100, validation_data=<keras_pre..., validation_steps=100, verbose=2, epochs=20)`
  del sys.path[0]


Epoch 1/20
 - 60s - loss: 0.6389 - acc: 0.6374 - val_loss: 0.4758 - val_acc: 0.8150
Epoch 2/20
 - 53s - loss: 0.4723 - acc: 0.7814 - val_loss: 0.3611 - val_acc: 0.8504
Epoch 3/20
 - 52s - loss: 0.4158 - acc: 0.8168 - val_loss: 0.3100 - val_acc: 0.8730
Epoch 4/20
 - 54s - loss: 0.3567 - acc: 0.8466 - val_loss: 0.2711 - val_acc: 0.8876
Epoch 5/20
 - 52s - loss: 0.3203 - acc: 0.8698 - val_loss: 0.2527 - val_acc: 0.8950
Epoch 6/20
 - 51s - loss: 0.2981 - acc: 0.8720 - val_loss: 0.2367 - val_acc: 0.8998
Epoch 7/20
 - 53s - loss: 0.2676 - acc: 0.8926 - val_loss: 0.2436 - val_acc: 0.8946
Epoch 8/20
 - 52s - loss: 0.2848 - acc: 0.8820 - val_loss: 0.2089 - val_acc: 0.9116
Epoch 9/20
 - 50s - loss: 0.2582 - acc: 0.8928 - val_loss: 0.2044 - val_acc: 0.9160
Epoch 10/20
 - 53s - loss: 0.2562 - acc: 0.8920 - val_loss: 0.1960 - val_acc: 0.9162
Epoch 11/20
 - 52s - loss: 0.2404 - acc: 0.9060 - val_loss: 0.1917 - val_acc: 0.9216
Epoch 12/20
 - 50s - loss: 0.2404 - acc: 0.9030 - val_loss: 0.1862 - val_a

# Predictions  On Model

In [0]:
train_generator.class_indices

{'cats': 0, 'dogs': 1}

In [0]:
import numpy as np
img_path = 'doggy.jpeg'
img = image.load_img(img_path, target_size=(IMG_WIDTH, IMG_HEIGHT))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
features

array([[0., 1.]], dtype=float32)

In [0]:
y_prob = model.predict(x) 
y_classes = y_prob.argmax(axis=-1)
y_classes

array([1])

In [0]:
import numpy as np
img_path = 'cat.jpg'
img = image.load_img(img_path, target_size=(IMG_WIDTH, IMG_HEIGHT))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
features

array([[1., 0.]], dtype=float32)

In [0]:
y_prob = model.predict(x) 
y_classes = y_prob.argmax(axis=-1)
y_classes

array([0])